In [162]:
from fileoperations.fileoperations import getFileNamesInDir
from symbtrdataextractor import symbtrreader, extractor
import os
import json
import pandas as pd 

import pdb

In [163]:
symbTr = 'acemasiran--yuruksemai--yuruksemai--bileydi_dil--itri'
symbtrtxtfile = os.path.join('../txt', symbTr + '.txt')
symbtrmu2file = os.path.join('../mu2', symbTr + '.mu2')

usuldict = json.load(open('./unittests/data/usul_extended.json','r'))

savefile = os.path.join('..', '..', 'symbtr_corrections', 'txt', symbTr + '.txt')

In [164]:
# extract symbtr data
txtdata = extractor.extract(symbtrtxtfile)[0]
mu2header = symbtrreader.readMu2Header(symbtrmu2file)[0]

if 'work' in txtdata.keys():
    mu2header['work'] = {'mu2_title':mu2header.pop('mu2_title')}
elif 'recording' in txtdata.keys():
    mu2header['recording'] = {'mu2_title':mu2header.pop('mu2_title')}
else:
    mu2header.pop('mu2_title')

# merge
data = extractor.merge(txtdata, mu2header)

    Some measures are skipped by the offsets
[33.08333, 44.16667, 53.08333, 58.08333, 68.08334, 69.08334, 70.08334, 71.08334, 72.08334, 73.08334, 78.08334, 79.08334, 93.08334, 96.08334, 97.08334, 98.08334]
    98.0: li   and vay   are in the same measure!
    96.0: li   and Ye are in the same measure!
    46.0: li   and ye are in the same measure!
    40.0: li   and Te are in the same measure!
    3.0: dir   and Yar  are in the same measure!
acemasiran--yuruksemai--yuruksemai--bileydi_dil--itri, 35, VOCAL_SECTION does not start on a measure: 3.666667
acemasiran--yuruksemai--yuruksemai--bileydi_dil--itri, 301, VOCAL_SECTION does not start on a measure: 33.08333
acemasiran--yuruksemai--yuruksemai--bileydi_dil--itri, 376, VOCAL_SECTION does not start on a measure: 40.83333
acemasiran--yuruksemai--yuruksemai--bileydi_dil--itri, 423, VOCAL_SECTION does not start on a measure: 46.83333
acemasiran--yuruksemai--yuruksemai--bileydi_dil--itri, 695, VOCAL_SECTION does not start on a measure: 78.0

In [165]:
for usul in usuldict.values():
    for uv in usul['variants']:
        if uv['mu2_name'] == data['usul']['mu2_name']:
            mertebe = uv['mertebe']
            zaman = uv['num_pulses']
            break


In [166]:
df = pd.read_csv(symbtrtxtfile, sep='\t')
for index, row in df.iterrows():
    # recompute the erroneous gracenotes with non-zero duration
    if row['Kod'] == 8 and row['Ms'] > 0:
        row['Pay'] = 0
        row['Payda'] = 0
        row['Ms'] = 0
        
    # recompute zaman and mertebe, if we hit kod 51
    if row['Kod'] == 51:
        zaman = row['Pay']
        mertebe = row['Payda']
        offset_incr = 0
    else:
        # compute offset
        offset_incr = 0 if row['Payda'] ==0 else float(row['Pay'])/row['Payda']*mertebe/zaman
    if index == 0:
        row['Offset'] = offset_incr
    else:
        prev_row = df.iloc[index-1]
        row['Offset'] = offset_incr + prev_row['Offset']
                
    # change null to empty string
    for key, val in row.iteritems():
        if pd.isnull(val):
            row[key] = ''
            
    # make sure that "Sira" column continues consecutively
    row['Sira'] = index+1
    
    # reassign
    df.iloc[index] = row
    

In [167]:
df.to_csv(savefile, sep='\t', index=False)

In [168]:
txtdata = extractor.extract(savefile)[0]
import numbers
if not (round(row['Offset']*10000)*0.0001).is_integer():
    print "Ends prematurely!"

    Some measures are skipped by the offsets
[33.0833333333, 44.1666666667, 53.0833333333, 58.0833333333, 68.0833333333, 69.0833333333, 70.0833333333, 71.0833333333, 72.0833333333, 73.0833333333, 78.0833333333, 79.0833333333, 93.0833333333, 96.0833333333, 97.0833333333, 98.0833333333]
    98.0: li   and vay   are in the same measure!
    96.0: li   and Ye are in the same measure!
    46.0: li   and ye are in the same measure!
    40.0: li   and Te are in the same measure!
    3.0: dir   and Yar  are in the same measure!
acemasiran--yuruksemai--yuruksemai--bileydi_dil--itri, 35, VOCAL_SECTION does not start on a measure: 3.66666666667
acemasiran--yuruksemai--yuruksemai--bileydi_dil--itri, 301, VOCAL_SECTION does not start on a measure: 33.0833333333
acemasiran--yuruksemai--yuruksemai--bileydi_dil--itri, 376, VOCAL_SECTION does not start on a measure: 40.8333333333
acemasiran--yuruksemai--yuruksemai--bileydi_dil--itri, 423, VOCAL_SECTION does not start on a measure: 46.8333333333
acemasi